In [1]:
# !jupyter serverextension enable voila --sys-prefix
# !voila --enable_nbextensions=True

In [2]:
# !pip install pytesseract

In [3]:
from ipywidgets import widgets, VBox
from PIL import ImageGrab, Image, ImageOps
import pyautogui
import win32gui
import time
import numpy as np
from IPython.display import clear_output
import re
import requests
import lxml.html as lh
import asyncio
import pandas as pd
import webbrowser
import pytesseract

In [4]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\paul.solomon\AppData\Local\Tesseract-OCR\tesseract.exe'

# PokerStars Spin & Go Tracker

In [5]:
tables = {}

In [6]:
# df = pd.DataFrame(index=['Player 1', 'Player 2', 'Player 3'], columns=['Stack Size'])
# blinds = [10,20]

In [7]:
class HaltableExecutor:

    def __init__(self, togglebutton):
        self._togglebutton = togglebutton
        self.blinds = []
        self.stacks = [0,0,0]

    async def my_code(self):
        # This is your user code
        active_table_no = 0
        button = ''
        while True:
            get_tables()
            if self._togglebutton.value:
                drop_table.disabled = True
                btn_ranges.layout.visibility = 'visible'
                
                if active_table_no in tables:
                    active_table = tables[active_table_no]
                    lbl_table.value = active_table

                    self.blinds = list(map(int, re.search(r'\d+\/\d+', active_table).group().split('/')))
                    lbl_blinds.value = 'Blinds are <b>%d</b>/<b>%d</b>' %(self.blinds[0], self.blinds[1])
                    
                    im = screenshot(active_table)
                    pot = get_pot(im)
                    if len(pot) > 0:
                        pot_number = re.search('\d+', pot).group()
                        lbl_pot.value = str(pot)
                    
                        if int(pot_number) == int(self.blinds[0]) + int(self.blinds[1]):
                            lbl_pot.value = 'calculating stacks'
                            self.stacks = get_stacks(im)
                            
                            if 0 in self.stacks[:3]:
                                if self.stacks[0] == 0:
                                    if self.stacks[1] == int(self.blinds[0]):
                                        button = '2'
                                    else: button = '3'
                                elif self.stacks[1] == 0:
                                    if self.stacks[2] == int(self.blinds[0]):
                                        button = '3'
                                    else: button = '1'
                                else:
                                    if self.stacks[0] == int(self.blinds[0]):
                                        button ='1'
                                    else: button = '2'
                            
                            else:                               
                                if self.stacks[3] == int(self.blinds[0]):
                                    button = '3'
                                elif self.stacks[3] == int(self.blinds[1]):
                                    button = '2'
                                else:
                                    button = '1'
                    
                            lbl_player1.value = 'Player 1: <b>%d</b>' % self.stacks[0]
                            lbl_player2.value = 'Player 2: <b>%d</b>' % self.stacks[1]
                            lbl_player3.value = 'Player 3: <b>%d</b>' % self.stacks[2]

                            lbl_button.value = '<b>Player %s</b> is the button' % button
                    
                    for lbl in [lbl_blinds, lbl_button, lbl_player1, lbl_player2, lbl_player3]:
                        lbl.layout.visibility = 'visible'
                    
#                     out.clear_output(wait=True)                    
#                     with out: display(df)
                    
                else: btn_on_off.value = False
                
            else:
                drop_table.disabled = False
                
                for lbl in [lbl_blinds, lbl_button, lbl_player1, lbl_player2, lbl_player3, btn_ranges]:
                    lbl.layout.visibility = 'hidden'
                    
                lbl_pot.value = ''
                
                self.stacks = [0,0,0]
                
#                 df.loc['Player 1'] = 0
#                 df.loc['Player 2'] = 0
#                 df.loc['Player 3'] = 0

#                 out.clear_output(wait=True)                
#                 with out: display(df)
                
                if len(tables) > 0:
                    active_table_no = list(tables.keys())[list(tables.values()).index(drop_table.value)]
                
            await asyncio.sleep(0.1)  # use this to temporarily give up control of the event loop to allow scheduling checkbox changes
        
    def start(self):
#         print('starting')
        asyncio.ensure_future(self.my_code())

In [8]:
def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))


def get_tables():
    tables.clear()
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for i in top_windows:
        if "tournament" and "table" in i[1].lower():
            tables[i[0]] = i[1]
    
    if len(tables) > 0:
        drop_table.options = list(tables.values())
    
    else: drop_table.options = ['No open tables']

def screenshot(window_title=None):
    if window_title:
        hwnd = win32gui.FindWindow(None, window_title)
        if hwnd:
            win32gui.SetForegroundWindow(hwnd)
            x, y, x1, y1 = win32gui.GetClientRect(hwnd)
            x, y = win32gui.ClientToScreen(hwnd, (x, y))
            x1, y1 = win32gui.ClientToScreen(hwnd, (x1 - x, y1 - y))
            im = pyautogui.screenshot(region=(x, y, x1, y1))
            return im
        else:
            print('Window not found!')
    else:
        im = pyautogui.screenshot()
        return im
    
def get_pot(im):
    area = (450, 190, 510, 210)
    cropped_im = ImageOps.invert(im.crop(area))
    return(pytesseract.image_to_string(cropped_im))

def get_stacks(im):
    area_1 = (480, 480, 520, 502)
    area_2 = (105, 155, 145, 178)
    area_3 = (810, 155, 850, 178)
    area_4 = (435, 370, 465, 395)

    
    cropped_ims = [ImageOps.invert(im.crop(area)) for area in [area_1,
                                                               area_2,
                                                               area_3,
                                                               area_4]]
    
    stacks = [pytesseract.image_to_string(cropped_im) for cropped_im in cropped_ims]

    return [int(x or 0) for x in stacks]

In [9]:
btn_on_off = widgets.ToggleButton(description='Tracking On/Off')

In [10]:
drop_table = widgets.Dropdown(options=[], 
                              value=None, # can use tuples of table_no, table_names here i fyou want
                              description='Select Table:',)

In [11]:
btn_ranges = widgets.Button(description='Get Ranges')

In [12]:
lbl_table = widgets.Label()
lbl_blinds = widgets.HTML()
lbl_player1 = widgets.HTML()
lbl_player2 = widgets.HTML()
lbl_player3 = widgets.HTML()
lbl_button = widgets.HTML()

lbl_pot = widgets.Label()

In [13]:
# stacks = widgets.HTML(df.style.set_table_attributes('class="table"').render())

In [14]:
# def on_button_clicked(click):
#     get_tables()

In [15]:
def on_select(change):
    lbl_table.value = drop_table.value

In [16]:
def on_get_ranges(click):
    s1, s2, s3 = exe.stacks
    sb, bb = exe.blinds
    
    url = ('https://www.holdemresources.net/nashicm?action=calculate&s1=' + 
           str(s1) + '&s2=' + str(s2) + '&s3=' + str(s3) +
           '&p1=70&p2=30&p3=0&sb=' + str(sb) + '&bb=' + str(bb) + '&ante=0')
    webbrowser.open(url)

In [17]:
out = widgets.Output()

In [18]:
# btn_tables.on_click(on_button_clicked)
drop_table.observe(on_select, 'value')
btn_ranges.on_click(on_get_ranges)

In [19]:
get_tables()

In [20]:
display(VBox([btn_on_off, 
              widgets.HTML('<br>'), 
              drop_table, 
              widgets.HTML('<br>'), 
              lbl_table,
              lbl_blinds,
#               out,
              lbl_player1,
              lbl_player2,
              lbl_player3,
              lbl_button,
              btn_ranges,
              lbl_pot
              ]))


In [21]:
exe = HaltableExecutor(btn_on_off)
exe.start()